# Network Update 2023

## Whats new? 

Its that time of the year again, lets see how many things needs to be changed this time.

In [63]:
from psycopg2 import connect
from psycopg2 import sql
from psycopg2.extras import execute_values
from pathlib import Path
import pandas as pd
import pandas.io.sql as pandasql
import configparser
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
CONFIG = configparser.ConfigParser()
CONFIG.read(str(Path.home().joinpath('db.cfg')))
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)
connection_engine = create_engine(
    URL.create(
        drivername = "postgresql",
        host = CONFIG['DBSETTINGS']['host'],
        database = CONFIG['DBSETTINGS']['database'],
        username = CONFIG['DBSETTINGS']['user'],
        password = CONFIG['DBSETTINGS']['password']))
cur = connection_engine.raw_connection().cursor()

In [72]:
import datetime
today = datetime.date.today()

first = today.replace(day=1)

In [73]:
query = '''
SELECT 
	accnb, year, accdate, stname1, stname2, stname3, traffictl, event1, event2, event3, district, submit_date
FROM collisions.acc
WHERE 
-- accyear within 3 years of this month and was submitted within this month
(((year::int >= extract(year from %(start_date)s - interval '3 years') AND 
  	year::int <= extract(year from %(start_date)s::date)) AND 
 (submit_date >= %(start_date)s AND submit_date <= %(end_date)s)) 
OR 
-- accdate within this month 
(accdate >= %(start_date)s AND accdate <= %(end_date)s)) 
-- event1, event2 or event3 within the selected code
AND (event1 IN ('50','51','52','53','54','55','56','57','58','59','60','61','62','63','64','65','66','23','24','25','26','27','28')
	OR event2 IN ('50','51','52','53','54','55','56','57','58','59','60','61','62','63','64','65','66','23','24','25','26','27','28')
	OR event3 IN ('50','51','52','53','54','55','56','57','58','59','60','61','62','63','64','65','66','23','24','25','26','27','28'))
'''
cur.execute(query, {'start_date': first, 'end_date': today})
header = cur.description
colnames = [desc[0] for desc in cur.description]
return_list = cur.fetchall()
df = pd.DataFrame(return_list, columns=colnames)
df

,accnb,year,accdate,stname1,stname2,stname3,traffictl,event1,event2,event3,district,submit_date
0,3002773877,2023,2023-12-02,GARDINER EXPRESSWAY W,PARK LAWN RD,10 m West of,10,01,51,None,None,2023-12-03
1,3002690298,2023,2023-11-21,CHRISTIE ST,TYRREL AV,None,02,61,None,None,None,2023-12-01
2,3002756234,2023,2023-11-29,NOTH QUEEN ST,EASTSIDE DR,25 m East of,02,61,99,55,None,2023-12-09
3,3002785632,2023,2023-12-03,LAKE SHORE BLVD W,SUPERIOR AV,None,01,01,63,None,None,NaT
4,3002714603,2023,2023-11-24,SCOTT ST,THE ESPLANADE,25 m North of,10,53,None,None,None,2023-12-09
5,3002634827,2023,2023-11-14,2 BLOOR ST E,BLOOR ST E,350 m South of,10,63,None,None,Toronto and East York,2023-12-09
6,3002780132,2023,2023-12-02,JANE ST,WILSON AV,None,01,27,None,None,None,NaT
7,3002748893,2023,2023-11-28,WARDEN AV,800 WARDEN AV,None,01,53,None,None,None,2023-12-09
8,3002593838,2023,2023-11-08,KINGSTON RD,EGLINTON AV E,172 m East of,10,66,None,None,None,2023-12-03
9,3002443371,2023,2023-10-21,STEELES AV E,BESTVIEW DR,10 m West of,10,52,None,None,None,2023-12-06


### Number of nodes that needs to be updated
only two!

In [29]:
sql = '''
select node_id from congestion.network_nodes
except 
select node_id from here.routing_nodes_23_4 
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
print(nodes)   

      node_id
0  30420390.0
1  30420392.0


### Number of links that needs to be updated
2990 links needs to be retired omg

In [30]:
sql = '''
select link_dir from congestion.network_links_22_2
except 
select link_dir from here.routing_streets_23_4 
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
print(nodes)   

         link_dir
0     1207662665F
1     1258933853T
2     1000005503T
3      995198819F
4       29568114T
...           ...
2985   949289551T
2986  1258735603F
2987  1258732905T
2988   993964936T
2989  1258776870T

[2990 rows x 1 columns]


### Number of segments that needs to be updated
1812 segment_ids needs be to retired and route............

In [31]:
sql = '''
with changed_links AS (
	select link_dir from congestion.network_links_22_2
	except 
	select link_dir from here.routing_streets_23_4)
	
select distinct segment_id from  congestion.network_links_22_2 
inner join changed_links using (link_dir)
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
print(nodes)   

      segment_id
0           6062
1           2335
2            652
3           5856
4           1560
...          ...
1807        2359
1808        3795
1809        1715
1810        3697
1811        4035

[1812 rows x 1 columns]


### Number of new and retired traffic signals
Quite a bunch of new traffic signals. We can deal with this after we recreated the network. 

In [32]:
sql = '''
select count(1) from gis.traffic_signal
where activationdate  >= '2022-01-01'
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()   

,count
0,43


# Updating the links

Create new network_links_23_4 table with newly routed segments for ones that contains outdated links

In [ ]:
'''
-- Create new links table except the segment_ids that are outdated

CREATE TABLE congestion.network_links_23_4 AS 
-- retired links
with changed_links AS (
	select link_dir from congestion.network_links_22_2
	except 
	select link_dir from here.routing_streets_23_4)
-- retired segments	
, changed_seg AS (
	select distinct segment_id, start_vid, end_vid 
	from  congestion.network_links_22_2
	inner join changed_links using (link_dir))
-- everything in network_links other than the retired segments
select      network_links_22_2.* 
from 		congestion.network_links_22_2
left join 	changed_seg using (segment_id)
where 		changed_seg is null ;

'''
'''
-- Insert routed results using new map version (23_4) for retired segments

with changed_links AS (
	select link_dir from congestion.network_links_22_2
	except 
	select link_dir from here.routing_streets_23_4)	
, changed_seg AS (
	select distinct segment_id, start_vid, end_vid 
	from  congestion.network_links_22_2 
	inner join changed_links using (link_dir))
 
 -- insert result to newly created network_links table
INSERT INTO congestion.network_links_23_4
SELECT segment_id, start_vid, end_vid, link_dir, routing.geom, round(st_length(st_transform(geom, 2952))::numeric, 2) as length
FROM   changed_seg
CROSS JOIN LATERAL pgr_dijkstra('SELECT id, source::int, target::int, st_length(st_transform(geom, 2952)) as cost 
				 	   			FROM here.routing_streets_23_4',  -- route using new map version's routing_streets
								start_vid, 
								end_vid)
INNER JOIN here.routing_streets_23_4 routing ON id = edge;
''';

### Check to see if all segments are inserted 
Returns nothing! 

In [33]:
sql = '''
select distinct segment_id from congestion.network_links_22_2
except
select distinct segment_id from congestion.network_links_23_4
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,segment_id


### Check to see if both returned the same number of segment_id
Both have 6558 segments. 

In [34]:
sql = '''

select count(distinct segment_id) from congestion.network_links_22_2
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,count
0,6558


In [35]:
sql = '''
select count(distinct segment_id) from congestion.network_links_23_4
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,count
0,6558


## Find if there are new traffic signals to add in 
25?!?!
Check them out on QGIS and see if they are legit. 
Checked: They are mostly legit other than one that are at a midblock and seems like a peds crossing.

In [36]:
sql = '''
-- Find segments that needs to be updated due to new traffic signals
with new_signal as (
	select px, ST_Transform(ST_buffer(ST_Transform(geom, 2952), 50), 4326) as geom 
	from gis.traffic_signal
	left join congestion.network_int_px_22_2 a using (px)
	where activationdate >= '2022-04-17' and a.px is null )

select count(distinct px)
from congestion.network_segments seg
join new_signal on ST_intersects(new_signal.geom, seg.geom)
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,count
0,25


## Find closest nodes for these new traffic signals 

Checked on QGIS, mostly look good in distance difference, except for one that has 40m difference (aka the one that is a ped cross). 

In [22]:
'''
-- Find segments that needs to be updated due to new traffic signals
with new_signal as (
	select px, ST_Transform(ST_buffer(ST_Transform(geom, 2952), 50), 4326) as bgeom, geom 
	from gis.traffic_signal
	left join congestion.network_int_px_22_2 a using (px)
	where activationdate >= '2022-04-17' and a.px is null )

, new_px AS (
	select distinct px, new_signal.geom as px_geom
	from congestion.network_segments seg
	join new_signal on ST_intersects(new_signal.bgeom, seg.geom))

-- Find new nodes for outdated nodes using nearest neighbour
			
select 	node_id, px, geom, px_geom, dist
from 	here.routing_nodes_23_4
CROSS JOIN LATERAL (SELECT z.px,
							px_geom, 
							(ST_transform(geom, 2952) <-> ST_Transform(z.px_geom, 2952)) as dist	
					FROM new_px z
					ORDER BY (geom <-> z.px_geom)
					LIMIT 1) nodes;
''';

## Route segments that was affected by new traffic signal

This query gathers all new nodes (find by nearest neighbour with traffic signal), as well as the start and end nodes of those affected segments (segments where new traffic signal lies). This creates a new table so its easier to QC on QGIS and make modifications.

In [ ]:
'''
--drop table 
CREATE TABLE congestion.network_segments_link_temp AS 
-- Find segments that needs to be updated due to new traffic signals
with new_signal as (
	select px, ST_Transform(ST_buffer(ST_Transform(geom, 2952), 50), 4326) as bgeom, geom as px_geom
	from gis.traffic_signal
	left join congestion.network_int_px_22_2 a using (px) -- where they are not already in old version
	where activationdate >= '2022-04-17' and a.px is null )
-- px that are new
, new_px AS (
	select distinct px, px_geom
	from congestion.network_segments seg
	join new_signal on ST_intersects(new_signal.bgeom, seg.geom))
-- segments affected by new signals
, affected_segment AS (
	select distinct start_vid, end_vid, segment_id
	from congestion.network_segments seg
	join new_signal on ST_intersects(new_signal.bgeom, seg.geom))
, affected_links AS (
	select link_dir 
	from affected_segment
	inner join congestion.network_links_23_4 USING (segment_id)
)
-- nodes that are closest to new traffic signal
, new_nodes AS (
	select 	node_id::int as node_id
	from 	 new_px
	CROSS JOIN LATERAL (SELECT node_id,
								geom, 
								(ST_transform(geom, 2952) <-> ST_Transform(px_geom, 2952)) as dist	
						FROM here.routing_nodes_23_4
						ORDER BY (geom <-> px_geom)
						LIMIT 1) nodes)
-- nodes for re-routing
, nodes AS (
	SELECT array_agg(node_id::int) as nodes_to_route 
	-- aggregate them into one array for many-to-many routing
	FROM	(select node_id 
			 FROM here.routing_nodes_23_4
			 where node_id in (30451170,852108714,30584642)
			 union 
			 select start_vid as node_id --start vid of retired segments
			 FROM affected_segment 
			 union 
			 select end_vid as node_id--end vid of retired segments
			 FROM affected_segment 
			 union
			 select distinct node_id 
			 FROM  new_nodes  
			 )a)
			
, results AS(
	SELECT results.*, link_dir, routing_grid.geom
	FROM nodes
	, LATERAL pgr_dijkstra('SELECT id, source::int, target::int, st_length(st_transform(geom, 2952)) as cost 
						   FROM here.routing_streets_23_4',
				nodes_to_route, nodes_to_route) results
	INNER JOIN here.routing_streets_23_4 routing_grid ON id = edge)	
, cleaned_results AS (
	SELECT 7113+row_number() over () as segment_id, 
				start_vid, 
				end_vid, 
				array_agg(link_dir order by path_seq) as link_set, 
				st_linemerge(st_union(s.geom)) as geom, 
				sum(cost) as length

	FROM results s
	LEFT OUTER JOIN (SELECT node_id FROM congestion.network_nodes_23
					 UNION 
					 SELECT unnest(nodes_to_route) FROM  nodes) network_nodes_23
					 ON node = node_id AND node != start_vid
	GROUP BY start_vid, end_vid
	HAVING COUNT(node_id) = 0 -- exclude routed results that went any nodes 
	order by start_vid, end_vid	)
, cleaned_links AS (
	select 	
		segment_id, 
		start_vid, 
		end_vid,
		link_dir, 
		geom, 
		round(ST_length(st_transform(geom, 2952))::numeric,2) as length,
		case when affected.link_dir is null then 'F' else 'T' end as used 
	from (select segment_id, start_vid, end_vid, unnest(link_set) as link_dir
		  from cleaned_results) a
	inner join here.routing_streets_23_4 using (link_dir)
	left join affected_links affected using (link_dir)
	order by segment_id)
	
select segment_id, start_vid, end_vid, ST_linemerge(ST_union(geom)) , sum(length), false, gis.direction_from_line(ST_linemerge(ST_union(geom))) as dir
from  cleaned_links
group by segment_id, start_vid, end_vid
having 'F' != ALL(array_agg(used));


'''